In [1]:
from langchain.graphs import Neo4jGraph
from langchain_openai import AzureChatOpenAI
from dotenv import load_dotenv
import os
from langchain.prompts import PromptTemplate
from langchain.chains import GraphCypherQAChain

load_dotenv()

NEO4J_URI = "neo4j+s://669a1604.databases.neo4j.io"
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = "aVhj0fbW8GaSkgk8aFjh3i2QmhIUWKnfnt2yCvJu-yk"

graph = Neo4jGraph(
    url=NEO4J_URI,
    username=NEO4J_USER,
    password=NEO4J_PASSWORD,
)

openai_api_key = os.getenv("AZURE_OPENAI_API_KEY")
openai_api_base = os.getenv("AZURE_OPENAI_ENDPOINT")
openai_api_version = os.getenv("AZURE_OPENAI_VERSION")
deployment_name = os.getenv("AZURE_DEPLOYMENT")

llm = AzureChatOpenAI(
    azure_endpoint=openai_api_base,
    openai_api_version=openai_api_version,
    azure_deployment=deployment_name,
    temperature=0,
    max_tokens=None,
)


C:\Users\emerg\AppData\Local\Temp\ipykernel_11932\221272689.py:14: LangChainDeprecationWarning: The class `Neo4jGraph` was deprecated in LangChain 0.3.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-neo4j package and should be used instead. To use it run `pip install -U :class:`~langchain-neo4j` and import as `from :class:`~langchain_neo4j import Neo4jGraph``.
  graph = Neo4jGraph(


In [2]:
schema_str = graph.schema

In [3]:
import pprint
pprint.pprint(schema_str)

('Node properties:\n'
 'InventoryLevel1 {name: STRING}\n'
 'InventoryLevel2 {name: STRING}\n'
 'InventoryLevel3 {name: STRING}\n'
 'InventoryLevel4 {name: STRING}\n'
 'GroupCompany {name: STRING}\n'
 'Relationship properties:\n'
 'HAS_AMOUNT {year: STRING, amount: FLOAT}\n'
 'The relationships:\n'
 '(:InventoryLevel1)-[:HAS_SUB_INVENTORY]->(:InventoryLevel2)\n'
 '(:InventoryLevel2)-[:HAS_SUB_INVENTORY]->(:InventoryLevel3)\n'
 '(:InventoryLevel2)-[:HAS_SUB_INVENTORY]->(:InventoryLevel4)\n'
 '(:InventoryLevel3)-[:HAS_SUB_INVENTORY]->(:InventoryLevel4)\n'
 '(:GroupCompany)-[:HAS_AMOUNT]->(:InventoryLevel4)')


In [4]:
def extract_nodes_by_label_from_markdown(md_path):
    with open(md_path, "r", encoding="utf-8") as f:
        md = f.read()
    
    import re

    # Patterns to extract each section (greedy until next heading or end of string)
    def extract_section(title):
        pattern = rf"### {re.escape(title)}\n((?:- .+\n)+)"
        match = re.search(pattern, md)
        if match:
            # Remove the leading "- " from each line and join
            items = [line[2:].strip() for line in match.group(1).strip().split("\n") if line.startswith("- ")]
            return "\n".join(items)
        else:
            return ""

    group_companies = extract_section("GroupCompany")
    inventorylevel1 = extract_section("InventoryLevel1")
    inventorylevel2 = extract_section("InventoryLevel2")
    inventorylevel3 = extract_section("InventoryLevel3")
    inventorylevel4 = extract_section("InventoryLevel4")

    return group_companies, inventorylevel1, inventorylevel2, inventorylevel3, inventorylevel4

# Example usage:
md_path = "kg_schema_overview.md"
group_companies, inventorylevel1, inventorylevel2, inventorylevel3, inventorylevel4 = extract_nodes_by_label_from_markdown(md_path)

# print("### GroupCompany")
# print(group_companies)
# print("\n### InventoryLevel1")
# print(inventorylevel1)
# print("\n### InventoryLevel2")
# print(inventorylevel2)
# print("\n### InventoryLevel3")
# print(inventorylevel3)
# print("\n### InventoryLevel4")
# print(inventorylevel4)


In [5]:
inventorylevel2

'Finished Goods\nInventory Machinery Spares\nOther Inventory\nRaw Material & Mfg. Componets\nStock in Trade\nStores and Spares\nTools\nWork-in-Progress'

In [6]:
inventory_type = inventorylevel1 + '\n' + inventorylevel2 + '\n' + inventorylevel3 + '\n' + inventorylevel4

In [7]:
inventory_type

'Inventories\nFinished Goods\nInventory Machinery Spares\nOther Inventory\nRaw Material & Mfg. Componets\nStock in Trade\nStores and Spares\nTools\nWork-in-Progress\nFinished Goods\nRaw Material & Mfg. Componet\nStock In Transit - Finised Goods\nStock In Transit - Raw Material\nStock In Transit - Traded Goods\nStock in Trade\nClg Stock - Construction W I P - B/s\nClg Stock - Finished Goods - B/s\nClg Stock - Mfg Component B/s\nClg Stock - Raw Material-B/s\nClg Stock - Wip - B/s\nClg Stock In Transit - FG B/s\nClg Stock In Transit - Rm - B/s\nClosing Stock- Gunny Bags\nConstruction Work In Progress\nFood & Beverages\nG-Stock Materiali Ricoperti CBN\nIn Transit - Stores & Spares\nInd-AS  Inventories\nInventorie Consumable Stores & Misc.\nInventory Dies\nInventory Machinery Spares-Imported\nInventory Manufactured Tools\nInventory Oils, Grease & Chemicals\nInventory Packing Material\nInventory Spares\nInventory Spares- Manual Adj\nInventory Stores- Manual Adj\nInventory Tools\nInventory To

In [8]:
group_companies

'AS\nAgri\nCE\nCONT SOUR\nCORP\nFES\nHO\nINTNL OPER\nMDS\nNON MRV\nPowerol\nSBU\nSOURCING\nSWARAJ\nSynergy\nTPDS\nTW'

In [9]:
NORMALIZE_PROMPT = """
You are a strict information extraction agent.
Your task is to extract and map relevant keywords from the user query to their exact forms as found in the provided lists.
You MUST return only the following JSON object and nothing else. Do not add explanations, notes, or preamble.

Group company names:
{group_companies}

Inventory type names:
{inventory_type}

Year values:
- FY23
- FY24

User query:
{user_query}

Return only this JSON object (use null if not found):
{{
  "company_name": "<matching group company name>",
  "inventory_type": "<matching inventory type>",
  "year": "<matching year>",
  "canonicalized_query": "<original query with all keywords replaced by their canonical KG forms>"
}}
"""


In [10]:
import json
import re
from pydantic import BaseModel, ValidationError
from typing import Optional

class NormalizationResult(BaseModel):
    company_name: Optional[str]
    inventory_type: Optional[str]
    year: Optional[str]
    canonicalized_query: Optional[str]

def normalize_entities(user_query):
    prompt = NORMALIZE_PROMPT.format(user_query=user_query, group_companies=group_companies, inventory_type=inventory_type)
    result = llm.invoke(prompt)
    content = result.content if hasattr(result, 'content') else str(result)
    print("Raw LLM output:", content)
    # Extract JSON robustly
    match = re.search(r"\{.*\}", content, re.DOTALL)
    json_str = match.group(0) if match else "{}"
    try:
        normalized = NormalizationResult.model_validate_json(json_str)
    except ValidationError as ve:
        print("Validation error:", ve)
        # Optionally, try to coerce to dict
        normalized = None
    return normalized

# Example usage:
user_query = "what are the value of Seed Potato Facility of the AS sector for year 2023?"
normalized = normalize_entities(user_query)
print("Normalized:", normalized)
if normalized:
    print("Canonicalized Query:", normalized.canonicalized_query)


Raw LLM output: {
  "company_name": "AS",
  "inventory_type": "WIP- Seed Potato Facility A/c",
  "year": "FY23",
  "canonicalized_query": "what are the value of WIP- Seed Potato Facility A/c of the AS sector for year FY23?"
}
Normalized: company_name='AS' inventory_type='WIP- Seed Potato Facility A/c' year='FY23' canonicalized_query='what are the value of WIP- Seed Potato Facility A/c of the AS sector for year FY23?'
Canonicalized Query: what are the value of WIP- Seed Potato Facility A/c of the AS sector for year FY23?


In [27]:
CYPHER_GENERATION_TEMPLATE = """
Task: Generate one or more Cypher queries that extract only first-level (direct) connections—nodes, relationships, and every property on them—for the inventories and/or group companies mentioned in the user’s question, using the schema below.

Instructions
- **Node labels allowed:** GroupCompany, InventoryLevel1, InventoryLevel2, InventoryLevel3, InventoryLevel4  
- **Relationship types allowed:** HAS_AMOUNT, HAS_SUB_INVENTORY  
- Always return every property for every matched node and relationship (e.g., name, year, amount).  
- If the question names a **group company, inventory, or year**, include an explicit `WHERE` filter for it.  
- Explicitly name relationship variables in `MATCH` and use those variables in `RETURN`.  
- Do not aggregate, create, or assume properties / labels / relationship types that aren’t in the schema.  
- When the question mentions a single inventory hierarchy (e.g., “Stores and Spares”), you may traverse that hierarchy with `HAS_SUB_INVENTORY*0..2`, but still return only direct relationships.  
- Provide **only** the Cypher query (no comments or explanations).

Schema:
{schema}

Rules
- Every variable returned **must** be matched in a `MATCH` clause.  
- Return all properties for those variables.

The question is:
{question}

Examples
1. Question: “What is the difference in Stock in trade count for the company 133-FES from FY24 to FY23?”  
   Cypher:
   MATCH (gc:GroupCompany {{name: "133-FES"}})-[r:HAS_AMOUNT]->(inv:InventoryLevel4 {{name: "Stock in trade"}})
   WHERE r.year IN ["FY23","FY24"]
   RETURN gc, r, inv

2. Question: “Why is the Stores and Spares inventory higher for Powerol in 2024 than in 2023?”  
   Cypher:
   MATCH (inv2:InventoryLevel2 {{name: "Stores and Spares"}})-[:HAS_SUB_INVENTORY*0..2]->(inv4:InventoryLevel4)
   MATCH (gc:GroupCompany {{name: "Powerol"}})-[r:HAS_AMOUNT]->(inv4)
   WHERE r.year IN ["2023","2024"]
   RETURN inv2, inv4, gc, r

3. Question: “What factors contribute to the difference in Finished Goods for SWARAJ for 2023 and 2024?”  
   Cypher:
   MATCH (inv2:InventoryLevel2 {{name: "Finished Goods"}})-[:HAS_SUB_INVENTORY*0..2]->(inv4:InventoryLevel4)
   MATCH (gc:GroupCompany {{name: "SWARAJ"}})-[r:HAS_AMOUNT]->(inv4)
   WHERE r.year IN ["2023","2024"]
   RETURN inv2, inv4, gc, r

4. Question: “What is the value of WIP- Seed Potato Facility A/c for the AS sector in FY23?”  
   Cypher:
   MATCH (gc:GroupCompany {{name: "AS"}})-[r:HAS_AMOUNT]->(inv:InventoryLevel4 {{name: "WIP- Seed Potato Facility A/c"}})
   WHERE r.year = "FY23"
   RETURN gc, r, inv

5. Question: “Which group companies are linked to Inventory Spares and what are the year and amount values?”  
   Cypher:
   MATCH (inv:InventoryLevel4 {{name: "Inventory Spares"}})<- [r:HAS_AMOUNT] - (gc:GroupCompany)
   RETURN inv, r, gc

6. Question: “Show all direct connections and their properties for the inventory item ‘Tools’.”  
   Cypher:
   MATCH (inv2:InventoryLevel2 {{name: "Tools"}})-[:HAS_SUB_INVENTORY]->(inv4:InventoryLevel4)
   RETURN inv2, inv4
   MATCH (inv4:InventoryLevel4 {{name: "Inventory Tools"}})<- [r:HAS_AMOUNT] - (gc:GroupCompany)
   RETURN inv4, r, gc
"""





cypher_prompt = PromptTemplate(
    input_variables=["query"],
    template=CYPHER_GENERATION_TEMPLATE,
)



# Create the GraphCypherQAChain with the prompt template and graph
chain = GraphCypherQAChain.from_llm(
    graph=graph,
    cypher_llm=llm,
    qa_llm=llm,
    cypher_prompt=cypher_prompt,
    verbose=True,
    allow_dangerous_requests=True,
    return_intermediate_steps=True
)

## Chain without the prompt template
chain2 = GraphCypherQAChain.from_llm(
    llm, graph=graph, verbose=True, allow_dangerous_requests=True
)


In [22]:
# 008-SWARAJ

In [29]:
user_query = "which inventory tools has the highest difference for AS from 2023 to 2024?"
normalized = normalize_entities(user_query)
print('INPUT QUERY NORMALIZED')
print(normalized.canonicalized_query)

query = normalized.canonicalized_query

# Run the chain with added context
result = chain.invoke({
    "query": query,
})

print('FINAL RESULT')
print(result)

Raw LLM output: {
  "company_name": "AS",
  "inventory_type": "Inventory Tools",
  "year": "FY24",
  "canonicalized_query": "which Inventory Tools has the highest difference for AS from FY23 to FY24?"
}
INPUT QUERY NORMALIZED
which Inventory Tools has the highest difference for AS from FY23 to FY24?


> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (inv2:InventoryLevel2 {name: "Tools"})-[:HAS_SUB_INVENTORY]->(inv4:InventoryLevel4)
MATCH (gc:GroupCompany)-[r:HAS_AMOUNT]->(inv4)
WHERE r.year IN ["FY23", "FY24"]
RETURN inv2, inv4, gc, r

Full Context:
[{'inv2': {'name': 'Tools'}, 'inv4': {'name': 'Inventory Tools'}, 'gc': {'name': 'AS'}, 'r': ({'name': 'AS'}, 'HAS_AMOUNT', {'name': 'Inventory Tools'})}, {'inv2': {'name': 'Tools'}, 'inv4': {'name': 'Inventory Tools'}, 'gc': {'name': 'AS'}, 'r': ({'name': 'AS'}, 'HAS_AMOUNT', {'name': 'Inventory Tools'})}, {'inv2': {'name': 'Tools'}, 'inv4': {'name': 'Inventory Tools'}, 'gc': {'name': 'FES'}, 'r': ({'name': 'FES'},

In [24]:
user_query = "What is the difference in Stock-in-trade count for the company 133-FES from FY24 to FY23?"

normalized = normalize_entities(user_query)
print('INPUT QUERY NORMALIZED')
print(normalized.canonicalized_query)

query = normalized.canonicalized_query

# Run the chain with added context
result = chain.run({
    "query": query,
})

print('FINAL RESULT')
print(result)

Raw LLM output: {
  "company_name": "FES",
  "inventory_type": "Stock in Trade",
  "year": "FY24",
  "canonicalized_query": "What is the difference in Stock in Trade count for the company 133-FES from FY24 to FY23?"
}
INPUT QUERY NORMALIZED
What is the difference in Stock in Trade count for the company 133-FES from FY24 to FY23?


> Entering new GraphCypherQAChain chain...


C:\Users\emerg\AppData\Local\Temp\ipykernel_11932\2815934324.py:10: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = chain.run({


Generated Cypher:
cypher
MATCH (gc:GroupCompany {name: "133-FES"})-[r1:HAS_AMOUNT]->(inv4:InventoryLevel4 {name: "Stock in trade"})
RETURN gc, r1, inv4

MATCH (gc:GroupCompany {name: "133-FES"})-[r2:HAS_AMOUNT]->(inv4FY23:InventoryLevel4 {name: "Stock in trade"})
WHERE r2.year = "FY23"
RETURN gc, r2, inv4FY23

MATCH (gc:GroupCompany {name: "133-FES"})-[r3:HAS_AMOUNT]->(inv4FY24:InventoryLevel4 {name: "Stock in trade"})
WHERE r3.year = "FY24"
RETURN gc, r3, inv4FY24



CypherSyntaxError: {code: Neo.ClientError.Statement.SyntaxError} {message: RETURN can only be used at the end of the query. (line 3, column 1 (offset: 114))
"RETURN gc, r1, inv4"
 ^}

In [26]:
user_query = "Why is the stores and spares increased for Powerol from 2023 to 2024?"

normalized = normalize_entities(user_query)
print('INPUT QUERY NORMALIZED')
print(normalized.canonicalized_query)

query = normalized.canonicalized_query

# Run the chain with added context
result = chain.run({
    "query": query,
})

print('FINAL RESULT')
print(result)

Raw LLM output: {
  "company_name": "Powerol",
  "inventory_type": "Stores and Spares",
  "year": "FY24",
  "canonicalized_query": "Why is the Stores and Spares increased for Powerol from FY23 to FY24?"
}
INPUT QUERY NORMALIZED
Why is the Stores and Spares increased for Powerol from FY23 to FY24?


> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (gc:GroupCompany {name: "Powerol"})<-[r:HAS_AMOUNT]-(inv4:InventoryLevel4)
RETURN gc, r, inv4

MATCH (inv2:InventoryLevel2)-[r1:HAS_SUB_INVENTORY]->(inv3:InventoryLevel3)
WHERE inv2.name = "Stores and Spares"
RETURN inv2, r1, inv3



CypherSyntaxError: {code: Neo.ClientError.Statement.SyntaxError} {message: RETURN can only be used at the end of the query. (line 3, column 1 (offset: 88))
"RETURN gc, r, inv4"
 ^}

In [ ]:
user_query = "Which inventory has the highest difference and the lowest difference in count for year 2023 and 2024"

normalized = normalize_entities(user_query)
print('INPUT QUERY NORMALIZED')
print(normalized.canonicalized_query)

query = normalized.canonicalized_query

# Run the chain with added context
result = chain.run({
    "query": query,
})

print('FINAL RESULT')
print(result)

In [ ]:
user_query = "which group company have lowest 'Finished products produced' count in 2023?"
normalized = normalize_entities(user_query)
print('INPUT QUERY NORMALIZED')
print(normalized.canonicalized_query)

query = normalized.canonicalized_query

# Run the chain with added context
result = chain.run({
    "query": query,
})

print('FINAL RESULT')
print(result)

In [ ]:
MATCH (g:GroupCompany)-[h:HAS_INVENTORY {year: 'FY24'}]->(i:InventoryType {name: 'Raw Material'})
RETURN g.name AS company_name, h.amount AS raw_material_amount
ORDER BY raw_material_amount DESC
LIMIT 1